In [1]:
# Dependencies
import requests, json, pprint, csv, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [44]:
# Reference path to raw data CSV 
csv_path = 'raw_data/athlete_events.csv'

# Read CSV into Pandas DataFrame
raw_olympics_df = pd.read_csv(csv_path)

raw_olympics_df.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN


In [45]:
list(raw_olympics_df.keys())

['ID',
 'Name',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'Team',
 'NOC',
 'Games',
 'Year',
 'Season',
 'City',
 'Sport',
 'Event',
 'Medal']

In [46]:
# Deleting unnecessary columns: 
#  - Name (because we have their individual ID numbers and humans are just numbers afterall)
#  - Team (because we care only about each country, not their special name)
#  - Games (because its information is contaned in Year & Season)

olympics_df = raw_olympics_df[[
 'ID',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'NOC',
 'Year',
 'Season',
 'City',
 'Sport',
 'Event',
 'Medal']]

olympics_df.head(10)

,ID,Sex,Age,Height,Weight,NOC,Year,Season,City,Sport,Event,Medal
0,1,M,24.0,180.0,80.0,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,M,23.0,170.0,60.0,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,M,24.0,NaN,NaN,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,M,34.0,NaN,NaN,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,F,21.0,185.0,82.0,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,F,21.0,185.0,82.0,NED,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,F,25.0,185.0,82.0,NED,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,F,25.0,185.0,82.0,NED,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,F,27.0,185.0,82.0,NED,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
9,5,F,27.0,185.0,82.0,NED,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN


In [47]:
# Reference path to raw data CSV 
csv_path = 'raw_data/noc_regions.csv'

# Read CSV into Pandas DataFrame
raw_NOC_df = pd.read_csv(csv_path)

raw_NOC_df.head(10)

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
5,ANG,Angola,NaN
6,ANT,Antigua,Antigua and Barbuda
7,ANZ,Australia,Australasia
8,ARG,Argentina,NaN
9,ARM,Armenia,NaN


In [48]:
NOC_df = raw_NOC_df[['NOC', 'region']]
NOC_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [49]:
# Merging the region's into the olympics df
olympics_df = pd.merge(olympics_df, NOC_df, how='left', on='NOC')
# Sorting by year
olympics_df = olympics_df.sort_values('Year')
# Capitolized the region column
olympics_df = olympics_df.rename(columns={'region':'Region'})
# Reset the index
olympics_df = olympics_df.reset_index(drop=True)
# Move the Region column to be next to NOC
olympics_df = olympics_df[[
 'ID',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'NOC',
 'Region',
 'Medal',
 'Sport',
 'Event',
 'Year',
 'Season',
 'City']]

olympics_df.head(10)

,ID,Sex,Age,Height,Weight,NOC,Region,Medal,Sport,Event,Year,Season,City
0,107607,M,19.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Pommelled Horse,1896,Summer,Athina
1,122526,M,19.0,NaN,NaN,FRA,France,Silver,Athletics,Athletics Men's Triple Jump,1896,Summer,Athina
2,122526,M,19.0,NaN,NaN,FRA,France,NaN,Athletics,Athletics Men's Long Jump,1896,Summer,Athina
3,12563,M,25.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Horse Vault,1896,Summer,Athina
4,12563,M,25.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Parallel Bars,1896,Summer,Athina
5,12563,M,25.0,NaN,NaN,GER,Germany,Gold,Gymnastics,"Gymnastics Men's Parallel Bars, Teams",1896,Summer,Athina
6,12563,M,25.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Horizontal Bar,1896,Summer,Athina
7,12563,M,25.0,NaN,NaN,GER,Germany,Gold,Gymnastics,"Gymnastics Men's Horizontal Bar, Teams",1896,Summer,Athina
8,12563,M,25.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Rings,1896,Summer,Athina
9,12563,M,25.0,NaN,NaN,GER,Germany,NaN,Gymnastics,Gymnastics Men's Pommelled Horse,1896,Summer,Athina


In [ ]:
output_path = 'olympics_df.csv'
olympics_df.to_csv(output_path, index=False)